In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
import pandas_gbq
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
UNION ALL
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
project_id = "perceptive-ivy-290216"
df_bq = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
GP=df_bq[((df_bq["GP"]=="Belgian Grand Prix") & (df_bq["Year"]==2024))]
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9961/2175524886.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GP["LapNumber"]=GP["LapNumber"].astype(int)


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
2166,0 days 00:57:12.422000,ALB,23,0 days 00:01:57.321000,1,1.0,NaT,NaT,NaT,0 days 00:00:52.352000,0 days 00:00:29.753000,NaT,0 days 00:56:42.783000,0 days 00:57:12.513000,333.0,198.0,221.0,297.0,False,MEDIUM,1.0,True,Williams,0 days 00:55:14.764000,NaT,1,10.0,False,,False,False,2024,Belgian Grand Prix
1808,0 days 00:59:03.705000,ALB,23,0 days 00:01:51.283000,2,1.0,NaT,NaT,0 days 00:00:31.152000,0 days 00:00:50.603000,0 days 00:00:29.528000,0 days 00:57:43.532000,0 days 00:58:34.135000,0 days 00:59:03.663000,NaN,196.0,219.0,314.0,True,MEDIUM,2.0,True,Williams,0 days 00:57:12.422000,NaT,1,10.0,False,,False,True,2024,Belgian Grand Prix
1809,0 days 01:00:53.905000,ALB,23,0 days 00:01:50.200000,3,1.0,NaT,NaT,0 days 00:00:30.836000,0 days 00:00:49.980000,0 days 00:00:29.384000,0 days 00:59:34.499000,0 days 01:00:24.479000,0 days 01:00:53.863000,332.0,196.0,218.0,313.0,True,MEDIUM,3.0,True,Williams,0 days 00:59:03.705000,NaT,1,10.0,False,,False,True,2024,Belgian Grand Prix
1810,0 days 01:02:44.215000,ALB,23,0 days 00:01:50.310000,4,1.0,NaT,NaT,0 days 00:00:31.057000,0 days 00:00:49.970000,0 days 00:00:29.283000,0 days 01:01:24.920000,0 days 01:02:14.890000,0 days 01:02:44.173000,325.0,196.0,218.0,NaN,False,MEDIUM,4.0,True,Williams,0 days 01:00:53.905000,NaT,1,10.0,False,,False,True,2024,Belgian Grand Prix
1811,0 days 01:04:34.938000,ALB,23,0 days 00:01:50.723000,5,1.0,NaT,NaT,0 days 00:00:30.919000,0 days 00:00:50.405000,0 days 00:00:29.399000,0 days 01:03:15.092000,0 days 01:04:05.497000,0 days 01:04:34.896000,349.0,192.0,218.0,315.0,False,MEDIUM,5.0,True,Williams,0 days 01:02:44.215000,NaT,1,10.0,False,,False,True,2024,Belgian Grand Prix


In [28]:
# GP=GP[GP["Driver"]!="GAS"]

In [29]:
fig=px.line(GP, x="LapNumber", y="Position", color='Driver',
#             markers=True,
        title="Race Position Ladder for the {} {}".format(year,gp),
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=1000, width=1200, template="plotly_dark",
             color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          },
        )

fig.update_layout(
    title_x=0.5,
    yaxis_range=[1, 21],
    margin=dict(l=10, r=5, t=55, b=20),
    font=dict(
        family="PT Sans Narrow",
        size=13,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1
    ),

    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),

    margin_pad=10
)
fig.update_yaxes(autorange="reversed")

# fig.update_layout(
    # updatemenus=[
    #     {
    #         "buttons": [
    #             {
    #                 "label": t.name,
    #                 "method": "restyle",
    #                 "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
    #             }
    #             for t in fig.data
    #         ]
    #     }
    # ]
# )

In [6]:
fig.write_html("{} {} Race Ladder.html".format(year,gp))

In [ ]:
#MULTI GP RACE LADDER

In [7]:
#2023
race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
# race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2023
for i in race_multi:    

    GP=df_bq[((df_bq["GP"]==i) & (df_bq["Year"]==year_multi))]
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP.head()
    fig=px.line(GP, x="LapNumber", y="Position", color='Driver',
            title="Race Position Ladder for the {} {}".format(year_multi,i),
            hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
            height=1000, width=1200, template="plotly_dark",
                color_discrete_map={
                 "ALB": "#37BEDD",
                 "ALO": "#358C75",
                 "BOT": "#C92D4B",
                 "DEV": "#5E8FAA",
                 "GAS": "#2293D1",
                 "HAM": "#6CD3BF",
                 "HUL": "#B6BABD",
                 "LAW": "#5E8FAA",
                 "LEC": "#F91536",
                 "MAG": "#B6BABD",
                 "NOR": "#F58020",
                 "OCO": "#2293D1",
                 "PER": "#3671C6",
                 "PIA": "#F58020",
                 "RIC": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "SAI": "#F91536",
                 "SAR": "#37BEDD",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA",
                 "VER": "#3671C6",
                 "ZHO": "#C92D4B"
            },
            )

    fig.update_layout(
        title_x=0.5,
        yaxis_range=[1, 21],
        margin=dict(l=10, r=5, t=55, b=20),
        font=dict(
            family="PT Sans Narrow",
            size=13,
            color="White"
        ),

        yaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
        ),

        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),

        margin_pad=10
    )
    fig.update_yaxes(autorange="reversed")
    fig.write_html("{} {} Race Ladder.html".format(year_multi,i))

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9961/4114248387.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9961/4114248387.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9961/4114248387.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
#LAP TIMES

In [39]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
GP=GP[GP["LapTime (s)"]<500]

In [40]:
fig_all_laps=px.scatter(GP, 
        x="LapNumber", 
        y="LapTime (s)", 
        color='Driver',
        title="All Lap Times for the {} {}".format(year,gp),
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=700,
          width=1200, 
        template="plotly_dark",
        color_discrete_map={
                 "AIT": "#0082fa",
                 "ALB": "#1E41FF",
                 "BOT": "#00D2BE",
                 "FIT": "#787878",
                 "GAS": "#2B4562",
                 "GIO": "#9B0000",    
                 "GRO": "#787878",               
                 "HAM": "#00D2BE",
                 "HUL": "#F596C8",
                 "KVY": "#2B4562",
                 "LAT": "#0082fa",
                 "LEC": "#DC0000",
                 "MAG": "#787878",
                 "NOR": "#FF8700",
                 "OCO": "#FFF500",
                 "PER": "#F596C8",
                 "RAI": "#9B0000",
                 "RIC": "#FFF500",
                 "RUS": "#0082fa",
                 "SAI": "#FF8700",
                 "STR": "#F596C8",
                 "VER": "#1E41FF",
                 "VET": "#DC0000",
          },
        )
fig_all_laps.update_layout(
    title_x=0.5,
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow",
    # margin=dict(l=5, r=5, t=35, b=5),
    ),
    updatemenus=[
        {
            "buttons": [
                {
                    "label": t.name,
                    "method": "restyle",
                    "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
                }
                for t in fig.data
            ]
        }
    ],
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 2
    ),

    margin_pad=10
)

In [9]:
fig_all_laps.write_html("{} {} All Lap Times.html".format(year,gp))

In [ ]:
#Multi GP Lap Times

In [11]:
#2023
race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
# 2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
# race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2023
for i in race_multi:
    GP=df_bq[((df_bq["GP"]==i) & (df_bq["Year"]==year_multi))]
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
    GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
    GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
    GP=GP[GP["LapTime (s)"]<500]
    fig_all_laps=px.scatter(GP, 
            x="LapNumber", 
            y="LapTime (s)", 
            color='Driver',
            title="All Lap Times for the {} {}".format(year_multi,i),
            hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
            height=700,
            width=1200, 
            template="plotly_dark",
            color_discrete_map={
                  "ALB": "#37BEDD",
                 "ALO": "#358C75",
                 "BOT": "#C92D4B",
                 "DEV": "#5E8FAA",
                 "GAS": "#2293D1",
                 "HAM": "#6CD3BF",
                 "HUL": "#B6BABD",
                 "LAW": "#5E8FAA",
                 "LEC": "#F91536",
                 "MAG": "#B6BABD",
                 "NOR": "#F58020",
                 "OCO": "#2293D1",
                 "PER": "#3671C6",
                 "PIA": "#F58020",
                 "RIC": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "SAI": "#F91536",
                 "SAR": "#37BEDD",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA",
                 "VER": "#3671C6",
                 "ZHO": "#C92D4B"
            },
            )
    fig_all_laps.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow",
        ),
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": t.name,
                        "method": "restyle",
                        "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
                    }
                    for t in fig.data
                ]
            }
        ],
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="White"
        ),

        yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 2
        ),

        margin_pad=10
    )
    fig_all_laps.write_html("{} {} All Lap Times.html".format(year_multi,i))

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9961/693301886.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9961/693301886.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9961/693301886.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [10]:
# GP=GP.sort_values(['LapNumber', "Driver"])
# GP.tail()

In [11]:
# fig2=px.scatter(GP,
#         x="LapNumber",
#         y="Position",
#         color='DriverNumber',
#         animation_frame="LapNumber",
#         title="Race Position Ladder for the {} {}".format(year,gp),
#         # animation_group='LapNumber',
#         range_x=[0,len(GP['LapNumber'].unique())+1],
#         text='Driver',
#         range_y=[0,20],
#         hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
#         height=1000, width=1800, template="plotly_dark",
#            color_discrete_map={
#                  "ALB": "#64C4FF",
#                  "ALO": "#229971",
#                  "BOT": "#52e252",
#                  "GAS": "#0093cc",
#                  "HAM": "#27F4D2",
#                  "HUL": "#B6BABD",
#                  "LEC": "#E80020",
#                  "MAG": "#B6BABD",
#                  "NOR": "#FF8000",
#                  "OCO": "#0093cc",
#                  "PER": "#3671C6",
#                  "PIA": "#FF8000",
#                  "RIC": "#6692FF",
#                  "RUS": "#27F4D2",
#                  "SAI": "#E80020",
#                  "SAR": "#37BEDD",
#                  "STR": "#229971",
#                  "TSU": "#6692FF",
#                  "VER": "#3671C6",
#                  "ZHO": "#52e252"
#           },
#          )


# fig2.update_yaxes(autorange="reversed")

# fig.update_traces(textposition='middle right')

# fig2.update_layout(
#     title_x=0.5,
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     font=dict(
#         family="PT Sans Narrow",
#         size=14,
#         color="White"
#     ),

#     yaxis = dict(
#     tickmode = 'linear',
#     tick0 = 0,
#     dtick = 1
#     ),

#     margin_pad=10
# )

# fig2.update(layout_coloraxis_showscale=False)
# # for x,y in zip(GP[GP["Driver"]=="SAI"].Driver, GP[GP["Driver"]=="SAI"].Position):
# #   for png in (Path.cwd().joinpath("f1").glob("*.png")):
# #     if str.split(str.split(str(png),".")[0],"/")[3]==x:
# #       image=str(png)
# #       fig2.add_layout_image(
# #           x=x,
# #           y=y+4,
# #           source=Image.open(image),
# #           xref="x",
# #           yref="y",
# #           sizex=5,
# #           sizey=5,
# #           xanchor="center",
# #           yanchor="middle",
# #       )
# fig2.show()

In [12]:
# fig2.write_html("{} {} Race Animation.html".format(year,gp))

# Fastest Driver in each lap

In [86]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["RK"] = GP_Rk.groupby("LapNumber")["LapTime"].rank(method="dense", ascending=True)
GP_FASTEST_LAPS=GP_Rk[GP_Rk["RK"]==1.0].sort_values("LapNumber")
GP_FASTEST_LAPS=GP_FASTEST_LAPS[["Time","Driver","Team","LapTime","LapNumber","Stint","PitOutTime","PitInTime","Sector1Time","Sector2Time","Sector3Time","SpeedI1","SpeedI2","TyreLife","TrackStatus","Position","Year","GP","RK"]]
GP_FASTEST_LAPS.head()

,Time,Driver,Team,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,TyreLife,TrackStatus,Position,Year,GP,RK
543798,0 days 00:58:40.015000,RUS,Mercedes,00:01:43.723000,1,1.0,NaT,NaT,NaT,0 days 00:00:31.645000,0 days 00:00:37.671000,190.0,264.0,1.0,12,1.0,2024,Canadian Grand Prix,1.0
108881,0 days 01:00:16.571000,RUS,Mercedes,00:01:36.556000,2,1.0,NaT,NaT,0 days 00:00:28.287000,0 days 00:00:30.597000,0 days 00:00:37.672000,198.0,264.0,2.0,1,1.0,2024,Canadian Grand Prix,1.0
74979,0 days 01:02:01.046000,MAG,Haas F1 Team,00:01:36.134000,3,1.0,NaT,NaT,0 days 00:00:27.936000,0 days 00:00:31.214000,0 days 00:00:36.984000,198.0,268.0,3.0,1,4.0,2024,Canadian Grand Prix,1.0
364215,0 days 01:03:35.039000,MAG,Haas F1 Team,00:01:33.993000,4,1.0,NaT,NaT,0 days 00:00:27.334000,0 days 00:00:29.992000,0 days 00:00:36.667000,215.0,265.0,4.0,1,4.0,2024,Canadian Grand Prix,1.0
129219,0 days 01:05:04.364000,VER,Red Bull Racing,00:01:33.859000,5,1.0,NaT,NaT,0 days 00:00:27.574000,0 days 00:00:30.293000,0 days 00:00:35.992000,200.0,267.0,5.0,1,2.0,2024,Canadian Grand Prix,1.0
